In [37]:
import numpy as np

In [38]:
class ExtendedKalmanFilter:
    def __init__(self, initial_state, initial_covariance, process_noise_covariance, measurement_noise_covariance, state_transition_function, observation_function):
        self.state_estimate = initial_state
        self.covariance_estimate = initial_covariance
        self.Q = process_noise_covariance
        self.R = measurement_noise_covariance
        self.state_transition_function = state_transition_function
        self.observation_function = observation_function

    def predict(self):
        # Predict step
        # Update state estimate using the state transition function
        self.state_estimate = self.state_transition_function(self.state_estimate) 
        # Update covariance estimate
        self.covariance_estimate = np.dot(np.dot(self.state_transition_jacobian(), self.covariance_estimate),
                                          self.state_transition_jacobian().T) + self.Q

    def update(self, observed_measurement):
        # Update step
        # Calculate the Kalman gain
        kalman_gain = np.dot(np.dot(self.covariance_estimate, self.observation_jacobian().T),
                             np.linalg.inv(np.dot(np.dot(self.observation_jacobian(), self.covariance_estimate),
                                                 self.observation_jacobian().T) + self.R))
        # Update state estimate
        self.state_estimate += np.dot(kalman_gain, (observed_measurement - self.observation_function(self.state_estimate)))
        # Update covariance estimate
        self.covariance_estimate = self.covariance_estimate - np.dot(np.dot(kalman_gain, self.observation_jacobian()),
                                                                     self.covariance_estimate)

    def state_transition_jacobian(self):
        # Implement the Jacobian matrix for the state transition function
        # This should be a 3x3 matrix for a 3-state system (position, velocity, acceleration)
        # For a simple linear system, the Jacobian is just an identity matrix
        return np.eye(8)

    def observation_jacobian(self):
        # Implement the Jacobian matrix for the observation function
        # This should be a 3x3 matrix for a 3-state system
        # For a simple linear measurement model, the Jacobian is also an identity matrix
        return np.eye(8)

In [39]:


# Define the initial state and covariance
initial_state = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
initial_covariance = np.eye(8)
process_noise_covariance = 0.01 * np.eye(8)  # Process noise covariance
measurement_noise_covariance = 0.1 * np.eye(8)  # Measurement noise covariance

# Define the process noise covariance matrix Q and measurement noise covariance matrix R
Q = np.eye(8)
R = np.eye(4)

# Define the state transition function and its Jacobian matrix
def state_transition_function(state):
    x, y, z, psi, u, v, w, r = state
    tau1, tau2, tau3, tau4 = (1, 0, 0, 0)
    # Given state transition function
    return np.array([
        np.cos(psi) * u - np.sin(psi) * v,
        np.cos(psi) * v + np.sin(psi) * u,
        w,
        r,
        (1315892839236077 * tau1) / 18014398509481984 - (1212595251356045 * u) / 9007199254740992 - 0.78804538217602188355025252290643 * r * v,
        (5149134226861935 * tau2) / 144115188075855872 - (8187638334133163 * v) / 72057594037927936 - 0.12541456625279259105718470889351 * r * u,
        (7356743338895695 * tau3) / 144115188075855872 + (1389688816717397 * w) / 72057594037927936,
        (6305868816807027 * r) / 18014398509481984 + (1135129035283523 * tau4) / 281474976710656 + 57.662298912638639724630477211478 * u * v
    ])

def state_transition_jacobian(state):
    # Given state transition Jacobian matrix
    # Replace the symbolic expressions with their corresponding values
    # Define this function appropriately using the symbolic values
    return np.array([[0, 0, 0, -np.cos(state[3]) * state[5] - np.sin(state[3]) * state[4], np.cos(state[3]), -np.sin(state[3]), 0, 0],
                     [0, 0, 0, np.cos(state[3]) * state[4] - np.sin(state[3]) * state[5], np.sin(state[3]), np.cos(state[3]), 0, 0],
                     [0, 0, 0, 0, 0, 0, 1, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1],
                     [0, 0, 0, 0, -0.13462511676066102328519491493353, -0.78804538217602188355025252290643 * state[7], 0, -0.78804538217602188355025252290643 * state[6]],
                     [0, 0, 0, 0, -0.12541456625279259105718470889351 * state[7], -0.11362630744822747874689383706936, 0, -0.12541456625279259105718470889351 * state[4]],
                     [0, 0, 0, 0, 0, 0, 1389688816717397 / 72057594037927936, 0],
                     [0, 0, 0, 0, 57.662298912638639724630477211478 * state[5], 57.662298912638639724630477211478 * state[4], 0, 0.35004603753424773371349942863162]])

# Define the observation function and its Jacobian matrix
def observation_function(state):
    return state[:4]

def observation_jacobian(state):
    return np.eye(4)


In [40]:
# Create an EKF instance
ekf = ExtendedKalmanFilter(initial_state, initial_covariance, process_noise_covariance,
                           measurement_noise_covariance, state_transition_function, observation_function)

# Perform EKF iterations (predict and update)
num_iterations = 100
for _ in range(num_iterations):
    ekf.predict()
    observed_measurement = np.array([10.0, 2.0, 1.0, 1.0])  # Simulated measurement
    ekf.update(observed_measurement)

# The final state estimate and covariance estimate
final_state_estimate = ekf.state_estimate
final_covariance_estimate = ekf.covariance_estimate
print(f"Final state estimate: {final_state_estimate}")
print(f"Final covariance estimate: {final_covariance_estimate}")

ValueError: shapes (8,8) and (4,) not aligned: 8 (dim 1) != 4 (dim 0)